# ML Estimation Full Observed Bounded Confidence Model
In this notebook we estimate the parameter $\epsilon$ in a BC model with full observations on edges and opinions.

The likelihood to be optimized is 

$\mathcal{L}(\epsilon) = \sum \log (\kappa \cdot s + (1- \kappa) \cdot (1 - s) ) $,

where $\kappa = \sigma(\rho \cdot (\epsilon - | \Delta X |))$ is the probability of having a positive interaction, and $s = 1$ if the interaction is positive, and $s = 0$ otherwise.

We maximise $\mathcal{L}$ with gradient descent.


In [1]:
import torch
import numpy as np
import torch.nn as nn

from scipy.special import expit as sigmoid
from scipy.special import logit
from time import time
from tqdm import tqdm

import sys
sys.path += ["../src"]
from simulator_opinion_dynamics import kappa_from_epsilon
import simulator_opinion_dynamics as sod
from initialize_model import EarlyStopping,RandomizeEpsilon,choose_optimizer


In [2]:
class Simple_BC_Estimation(nn.Module):
    
    def __init__(self, parameters0, X, edges):
        
        super().__init__()
        
        # epsilon0 is the initialization of epsilon
        epsilon0, rho = parameters0
        self.rho = rho
        u,v,s,t = uvst = sod.convert_edges_uvst(edges)
        # store the matrix of the differences of X and update it at each time
        self.diff_X = X[t,u] - X[t,v]
        # optimize theta, that is the logit of 2 * epsilon (this is useful to bound epsilon in [0, 0.5])
        theta = torch.tensor([logit(2 * epsilon0)], requires_grad = True)
        self.theta = nn.Parameter(theta)
        
    def forward(self):
        epsilon = torch.sigmoid(self.theta) / 2
        # at each step compute the probability of having positive interactions from the current epsilon
        kappa = kappa_from_epsilon(epsilon, self.diff_X, self.rho)
        return kappa
    
    def neg_log_likelihood_function(kappa, s, t_minibatch):
        # compute the negative log likelihood, that is the loss to be optimized
        return -(torch.sum(torch.log((kappa * s) + ((1 - kappa) * (1 - s)))))
    
    def neg_log_likelihood_function_minibatch(kappa, s, t_minibatch):
        # same as before, when using minibatching
        return -(torch.sum(torch.log((kappa * s) + ((1 - kappa) * (1 - s)))[t_minibatch]))
    

In [3]:
def gradient_descent_simple_BC(X, edges, rho, num_epochs, epsilon0 = 0.25, optimizer_name = "adam",
                               lr = 0.05, hide_progress = True, minibatch_size = 0, seed = None,
                               early_stopping_kw = {"patience": 20, "min_delta": 1e-5, 
                                                    "min_epochs": 20, "long_run_delta": 1e-5, 
                                                    "long_run_diff":10, "long_run_patience": 5}):
    if seed is not None:
        np.random.seed(seed)
    u,v,s,t = uvst = sod.convert_edges_uvst(edges)
    
    T,N = X.shape
    
    model_class = Simple_BC_Estimation
    model = model_class((epsilon0, rho), X, edges)
    if minibatch_size == 0:
        loss_function = model_class.neg_log_likelihood_function
    if minibatch_size > 0:
        loss_function = model_class.neg_log_likelihood_function_minibatch
    
    
    early_stopping = EarlyStopping(**early_stopping_kw)
    optimizer = choose_optimizer(optimizer_name, lr, model)
    
    # store all the estimates of epsilon and the loss at each epoch
    history = {"epsilon": [epsilon0], "loss": []}
    
    t0 = time()
    for epoch in tqdm(range(num_epochs), disable = hide_progress):
        t_minibatch = torch.randperm(T-1)[:minibatch_size]
        
        kappa = model()
        loss = loss_function(kappa, s, t_minibatch)
        
        loss.backward()
        optimizer.step()
        
        history["epsilon"].append(sigmoid(model.theta.item()) / 2)
        history["loss"].append(loss.item())
        
        optimizer.zero_grad()
        
        if epoch > early_stopping_kw["min_epochs"]:
            early_stopping(history["epsilon"][-3], history["epsilon"][-2], history["epsilon"][-1], epoch)
        if early_stopping.early_stop:
            break
            
    t1 = time()
    history["time"] = t1 - t0
    
    return history

In [4]:
N, T, edge_per_t = 100, 256, 4
evidences_per_t = 4
epsilon, mu, rho = 0.12, 0.4, 16

X, edges, evidences = sod.simulate_BC(N, T, edge_per_t, evidences_per_t, (epsilon, mu, rho), seed = 2006)

In [7]:
history = gradient_descent_simple_BC(X, edges, rho, num_epochs = 100, epsilon0 = 0.25, optimizer_name = "adam",
                           lr = 0.05, seed = 2912)
    